In [1]:
!pip install fredapi

  Using cached fredapi-0.5.2-py3-none-any.whl.metadata (5.0 kB)
Using cached fredapi-0.5.2-py3-none-any.whl (11 kB)


In [2]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
import fredapi as fa